## 1. Setup

Before we begin, let's make sure your environment is set up correctly. We'll start by installing the necessary Python packages.

### Installing Required Packages

To get started, you'll need to install a few Python libraries. Run the following command to install them:

In [21]:
%pip install langchain langgraph langgraph-checkpoint-sqlite langchain_community requests termcolor wikipedia arxiv

Note: you may need to restart the kernel to use updated packages.


## 2. Configuring a Simple Model

In this section, we configure the machine learning model that our agent will use to process tasks. The `ModelService` class manages the interaction with the model (in this case, "llama3.1:8b-instruct-fp16"), allowing the agent to handle tasks such as listing VMs and retrieving details.

### Model Configuration

We initialize the `ModelService` with a specific model configuration, including parameters such as model endpoint, temperature (for controlling randomness), and others. This step enables our agent to perform model-based tasks using the provided configuration.

In [22]:
from services.model_service import ModelService

# Initialize the service with the model configuration
ollama_service = ModelService(model="llama3.1:8b-instruct-fp16")

## 3. State Management

State management is an essential concept when working with agents. The "state" of an agent refers to its current condition or the information it has at any given time. For instance, if an agent is working through a list of tasks, its state might include which tasks have been completed, which are in progress, and which are yet to be started. Without proper state management, an agent might lose track of its progress, repeat tasks, or skip important steps. In this notebook, you'll learn how to manage an agent's state effectively, ensuring that it operates smoothly and efficiently.

### Implementation of State Management

To implement state management for our ReAct agent, we define a structured data model (`AgentGraphState`) that holds all the relevant information the agent needs to function effectively. This model includes:
- **Input:** The current command or task that the agent is working on.
- **Response:** The outputs or actions the agent has generated.

Additionally, we provide a utility function, `update_state`, which allows for updating specific elements of the agent's state. This function ensures that the state is consistently and accurately maintained, which is critical for the agent to operate effectively. By checking for the existence of keys before updating, the function helps prevent errors and maintains the integrity of the state.

Together, these components form the backbone of the agent's state management system, enabling it to manage complex workflows and adapt to changes dynamically.

In [23]:
from langgraph.graph.message import add_messages
from typing import Annotated, TypedDict, Any


class AgentGraphState(TypedDict):
    input: str
    researcher_response: Annotated[list, add_messages]
    search_response: Annotated[list, add_messages]


def update_state(state: AgentGraphState, key: str, value: Any):
    """
    Update the state of the agent. Warn if the key doesn't exist.
    """
    if key in state:
        state[key] = value
    else:
        print(f"Warning: Attempting to update a non-existing state key '{key}'.")

## 4. Creating the agents

In [24]:

from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)

# Adding the search tool to the list of available tools
researcher_tools = [arxiv, wiki]

In [25]:
from agent.react_agent import ReactAgent

def researcher_node_function(state: AgentGraphState):
    researcher_agent = ReactAgent(
        state=state,
        role="researcher",
        tools=researcher_tools,
        ollama_service=ollama_service,
    )

    return researcher_agent.react(user_request=state["input"])

In [26]:
from langchain_community.tools import DuckDuckGoSearchRun

# Initialize DuckDuckGo Search Tool
duckduckgo_search = DuckDuckGoSearchRun()

search_tools = [duckduckgo_search]

In [27]:
def search_node_function(state: AgentGraphState):
    search_agent = ReactAgent(
        state=state,
        role="search",
        tools=search_tools,
        ollama_service=ollama_service,
    )

    return search_agent.react(user_request=state["input"])

## 5. Creating and Compiling the Workflow Graph

In this step, we build the workflow graph that represents the agent's process. This graph outlines the sequence of operations, including task execution, validation, and decision-making.

### Key Components:
- **Node Definitions**: Each node in the graph represents a step in the workflow, such as invoking the ReAct agent.
  
- **Edge Definitions**: Edges define the flow between nodes, determining how the agent progresses through the tasks and validation steps.

- **Workflow Compilation**: Once the graph is defined, it is compiled into a workflow that can be executed.

By constructing and compiling this workflow graph, we ensure that the PM agent operates in a structured and efficient manner, handling tasks and making decisions in a logical sequence.

In [28]:
from langgraph.graph import StateGraph, START, END


def create_graph() -> StateGraph:
    """
    Create the state graph by defining nodes and edges.

    Returns:
    - StateGraph: The compiled state graph ready for execution.
    """
    graph = StateGraph(AgentGraphState)

    # Add nodes
    graph.add_node("researcher_agent", researcher_node_function)
    graph.add_node("search_agent", search_node_function)

    # Define the flow of the graph
    graph.add_edge(START, "researcher_agent")
    graph.add_edge("researcher_agent", "search_agent")
    graph.add_edge("search_agent", END)

    return graph

### Initializing Sqlite Persistence for Graph State

In this cell, we define and use a method to initialize an `SqliteSaver` instance from the `langgraph.checkpoint.sqlite` module. The `SqliteSaver` class allows the graph state to be persisted in an SQLite database, which is more durable and suitable for applications requiring longer-term storage compared to an in-memory solution.

The `from_conn_stringx` method is defined as a class method that takes a connection string as input, creates a connection to the SQLite database using `sqlite3.connect`, and then returns an `SqliteSaver` instance using this connection. This method simplifies the creation of an `SqliteSaver` instance directly from a connection string.

This approach is particularly useful for ensuring that the state of the `StateGraph` is saved to a local or memory-based SQLite database, enabling the retention of context across multiple interactions in AI-driven applications.

In [29]:
from langgraph.checkpoint.sqlite import SqliteSaver
import sqlite3


def from_conn_stringx(
    cls,
    conn_string: str,
) -> "SqliteSaver":
    return SqliteSaver(conn=sqlite3.connect(conn_string, check_same_thread=False))


SqliteSaver.from_conn_stringx = classmethod(from_conn_stringx)

memory = SqliteSaver.from_conn_stringx(":memory:")

## 6. Executing the Workflow

With the workflow graph compiled, the final step is to execute the workflow. This involves providing the agent with an input query, such as a request to generate a VM migration plan, and allowing the workflow to run through its defined sequence.

### Creating and Running the Workflow

In this step, we create and execute the PM agent's workflow to process a set of tasks.

- **Graph Creation**: 
  - We first create the workflow graph using `create_graph()` and compile it with a memory-based checkpoint.
  - The compiled workflow will manage the task execution, validation, and feedback handling.

- **Workflow Parameters**:
  - We define the number of iterations (`iterations = 10`), set verbose mode to `True`, and configure the thread ID.
  - A query containing three tasks (VM details retrieval, migration plan creation, and migration start) is provided as input.

- **Workflow Execution**:
  - The workflow is executed using `workflow.stream()`, and it processes each task sequentially.
  - Depending on the state of the workflow, feedback or task responses are printed to track progress.

This step runs the agent through the defined tasks and prints the state changes for each event in the workflow.

In [30]:
# Create the graph and compile the workflow
graph = create_graph()
workflow = graph.compile(checkpointer=memory, interrupt_before=["search_agent"])
print("Graph and workflow created.")

# Define workflow parameters
iterations = 10
verbose = True
config = {"configurable": {"thread_id": "1"}}

query = "Who's the USA's President?"
dict_inputs = {"input": query}
limit = {"recursion_limit": iterations}

# Execute the workflow and print state changes
for event in workflow.stream(dict_inputs, config):
    if verbose:
        print("\nEvent:", event)
    else:
        print("\n")

Graph and workflow created.
<|start_header_id|>user<|end_header_id|>

Who's the USA's President?<|eot_id|>


<|start_header_id|>assistant<|end_header_id|>

{
    "thought": "To find out who the current US President is, I can use the Wikipedia tool to search for the relevant information.",
    "action": "wikipedia",
    "action_input": {
        "query": {
            "title": "President of the United States",
            "description": "",
            "type": ""
        }
    }
}<|eot_id|>
<|python_tag|>wikipedia.call({'query': {'title': 'President of the United States', 'description': '', 'type': ''}})
<|eom_id|>
<|start_header_id|>ipython<|end_header_id|>

{
    "observation": "Error executing tool wikipedia: 1 validation error for WikipediaQueryInput\nquery\n  str type expected (type=type_error.str)"
}<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>

{
    "thought": "The previous attempt failed due to a formatting issue. I will try again with the correct input format.",
    "action": "wikipedia",
    "action_input": {
        "query": "President of the United States"
    }
}<|eot_

In [33]:
import json
from langchain_core.messages.ai import AIMessage
from langchain_core.messages import SystemMessage

def custom_serializer(obj):
    if isinstance(obj, AIMessage):
        return {
            "content": obj.content,
            "id": obj.id,
            # Add other attributes as needed
        }
    elif isinstance(obj, SystemMessage):
        return {
            "content": obj.content,
            "id": obj.id,
            # Add other attributes as needed
        }
    # Add other custom class serializations here
    else:
        return str(obj)  # Fallback to string conversion if not recognized


state_snapshot = workflow.get_state(config)

print(json.dumps(state_snapshot.values, indent=4, default=custom_serializer))

print(state_snapshot.next)

{
    "input": "Who's the USA's President?",
    "researcher_response": [
        {
            "content": "I have the answer: Page: President of the United States\nSummary: The president of the United States (POTUS) is the head of state and head of government of the United States of America. The president directs the executive branch of the federal government and is the commander-in-chief of the United States Armed Forces.",
            "id": "9d8e3289-58ee-4101-a5e0-f7c48b7f1e79"
        }
    ],
    "search_response": []
}
('search_agent',)


In [34]:
# Execute the workflow and print state changes
for event in workflow.stream(None, config):
    if verbose:
        print("\nEvent:", event)
    else:
        print("\n")

<|start_header_id|>user<|end_header_id|>

Who's the USA's President?<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>

{
    "thought": "To find out who the current president of the United States is, I can use a search tool to look up the information.",
    "action": "duckduckgo_search",
    "action_input": {
        "query": {
            "title": "Current President of the USA",
            "description": "search query to find the current president of the USA",
            "type": "string"
        }
    }
}<|eot_id|>
<|python_tag|>duckduckgo_search.call({'query': {'title': 'Current President of the USA', 'description': 'search query to find the current president of the USA', 'type': 'string'}})
<|eom_id|>
<|start_header_id|>ipython<|end_header_id|>

{
    "observation": "Error executing tool duckduckgo_search: 1 validation error for DDGInput\nquery\n  str type expected (type=type_error.str)"
}<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>

{
    "thought": "The previo